In [57]:
import pandas as pd
import numpy as np
import requests

In [58]:
r1 = requests.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
visits_data = r1.json()
visits_df = pd.DataFrame(visits_data)

# Исключение ботов
visits_df = visits_df[visits_df['user_agent'] != 'bots']

# Оставляем только последний визит пользователя
visits_df['datetime'] = pd.to_datetime(visits_df['datetime'])
visits_df = visits_df.sort_values(by=['datetime']).drop_duplicates(subset=['visit_id'], keep='last')

r2 = requests.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
data_regs = r2.json()
regs_df = pd.DataFrame(data_regs)

regs_df['datetime'] = pd.to_datetime(regs_df['datetime'])
regs_df['date_group'] = regs_df['datetime'].dt.date
visits_df['date_group'] = visits_df['datetime'].dt.date

regs_grouped = regs_df.groupby(['date_group', 'platform']).size().reset_index(name='registrations')
visits_grouped = visits_df.groupby(['date_group', 'platform']).size().reset_index(name='visits')

regs_grouped['date_group'] = pd.to_datetime(regs_grouped['date_group'])
visits_grouped['date_group'] = pd.to_datetime(visits_grouped['date_group'])

# Объединение данных с inner join
merged_df = pd.merge(regs_grouped, visits_grouped, on=['date_group', 'platform'], how='inner')

# Приведение типов
merged_df['registrations'] = merged_df['registrations'].astype(int)
merged_df['visits'] = merged_df['visits'].astype(int)

# Расчет conversion без округления
merged_df['conversion'] = np.where(
    merged_df['visits'] == 0, 0, 
    merged_df['registrations'] / merged_df['visits'] * 100
)

# Упорядочивание колонок
merged_df = merged_df[['date_group', 'platform', 'visits', 'registrations', 'conversion']]

# Сохранение JSON
merged_df.to_json('./conversion.json')